#  **Practice. Computer vision; convolutional neural networks**

## Convolutional neural network

### Useful links (tutorials)

https://www.tensorflow.org/tutorials/images/classification

https://developers.google.com/machine-learning/practica/image-classification/convolutional-neural-networks


### Useful links (layers)

https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Add

In [ ]:
! pip install livelossplot

### **Exercise**: downolad the cifar10 dataset, reproduce one fully-connected architecture from the homework, train


Download the dataset

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
(train_images_full, train_labels_full), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
train_images_full, test_images = train_images_full / 255.0, test_images / 255.0

In [ ]:
print("Train size = %i, test_size = %i"%(len(train_images_full),len(test_images)))

Look at the images from the training set

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(15,15))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images_full[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels_full[i][0]])
plt.show()

Split traininig set into training and validation sets (1% of the training set)


In [ ]:
train_images, validation_images, train_labels, validation_labels = ### your code here

In [ ]:
len(validation_images)

Create a dataset and preprocess it for further training:


In [ ]:
batch_size = 512

train_dataset = ### your code here

val_dataset = ### your code here


Build a Sequential Model like in the first part of the class. 

Use a Flatten layer to represent images as vectors: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten


In [ ]:
from tensorflow.keras import layers


input_shape = train_images.shape[-3:]

dense_model = tf.keras.Sequential([
    ### your code here
])

Compile and train the model.

This time we'll use **SparseCategoricalCrossentropy**, thus we don't need one hot encoding.

https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy

In [ ]:
dense_model.compile(
    optimizer='sgd',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [ ]:
dense_model.summary()

In [ ]:
from livelossplot import PlotLossesKeras


In [ ]:
dense_model_history = dense_model.fit(train_dataset, validation_data=val_dataset, epochs=100, callbacks=[PlotLossesKeras()], verbose=False)

### **Exercise**: add convolutions


In [ ]:
cnn_model = tf.keras.Sequential([
    ### your code here
])

cnn_model.compile(
    optimizer='sgd',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [ ]:
cnn_model.summary()

In [ ]:
cnn_model_history = cnn_model.fit(train_dataset, validation_data=val_dataset, epochs=100, callbacks=[PlotLossesKeras()], verbose=False)

### **Exercise**: add augmentations to the training dataset, look at the images

https://www.tensorflow.org/tutorials/images/data_augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal"),
  # layers.RandomRotation(0.1),
])


In [ ]:
import numpy as np

augmented_train_dataset = train_dataset.map(
  lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
data_augmentation_vis = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.1),
])

image = tf.cast(tf.expand_dims(train_images_full[6], 0), tf.float32)

plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = data_augmentation_vis(image, training=True)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_image[0])
  plt.axis("off")

In [ ]:
sample_x, sample_y = next(iter(augmented_train_dataset))

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(sample_x[i])
  plt.axis("off")

In [ ]:
cnn_model_augmented_train = tf.keras.Sequential([
    layers.Input(shape=input_shape), 
    layers.Conv2D(16, (3, 3), activation='relu', padding='same'),          
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(10)
])

cnn_model_augmented_train.compile(
    optimizer='sgd',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [ ]:
cnn_model_augmented_train_history = cnn_model_augmented_train.fit(augmented_train_dataset, validation_data=val_dataset, epochs=100, callbacks=[PlotLossesKeras()], verbose=False)

### Comparison of the training plots


In [ ]:
acc_dense = dense_model_history.history['accuracy']
val_acc_dense = dense_model_history.history['val_accuracy']

acc_cnn = cnn_model_history.history['accuracy']
val_acc_cnn = cnn_model_history.history['val_accuracy']

acc_cnn_aug = cnn_model_augmented_train_history.history['accuracy']
val_acc_cnn_aug = cnn_model_augmented_train_history.history['val_accuracy']

epochs_range = range(100)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc_dense, label='Dense Training Accuracy')
plt.plot(epochs_range, acc_cnn, label='CNN Training Accuracy')
plt.plot(epochs_range, acc_cnn_aug, label='CNN Augmented Training Accuracy')
plt.legend(loc='lower right')
plt.title('Training Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, val_acc_dense, label='Dense Validation Accuracy')
plt.plot(epochs_range, val_acc_cnn, label='CNN Validation Accuracy')
plt.plot(epochs_range, val_acc_cnn_aug, label='CNN Augmented Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Validation Accuracy')
plt.show()

### **Task**: add skip-connection

https://www.tensorflow.org/api_docs/python/tf/keras/Model

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Add
